In [1]:
import deeppavlov

In [30]:
import numpy as np
import pandas as pd
import importlib
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Embedding, Dropout, LSTM, Input,  Bidirectional, Conv1D, MaxPooling1D, GRU, CuDNNLSTM, Lambda, Layer
from keras.layers.core import Dense, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from sklearn.model_selection import train_test_split
import functions_train
import functions_for_intensity_events
from keras.regularizers import l1, l1_l2, l2
from keras.metrics import categorical_accuracy
from keras.callbacks import EarlyStopping

from keras import backend as K
import importlib
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, MinMaxScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, precision_score, f1_score, recall_score, classification_report

In [ ]:

elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [33]:
import tensorflow as tf
sess = tf.Session()
sess.run(tf.global_variables_initializer())
model =  elmo_model(tf.squeeze(tf.cast(['apple', 'orange', 'bannana', 'strowberry'], tf.string)), signature="default", as_dict=True)
a = sess.run(model)

2019-01-18 16:34:59.906 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


In [5]:
#prepare dataset

columns = ['CODE_GROUP', 'DUBP_CODE_NUM', 'ID']
data_events, list_criterias = functions_for_intensity_events.prepare_data_labels('main_ori_aero_advice_table.xlsx', columns)


In [6]:
data_events['CODE_GROUP_NEW'][data_events['CODE_GROUP_NEW'] == 'NAN']  = data_events['NEW_GROUP'][data_events['CODE_GROUP_NEW'] == 'NAN']

In [9]:
binazer = LabelBinarizer()
criterias_group = binazer.fit_transform(data_events['CODE_GROUP_NEW'].astype(str))

criterias_group = pd.DataFrame(criterias_group, columns = binazer.classes_)

In [15]:
X_toc = data_events['CLEAN_TEXT'].apply(lambda x: x[:45])
X_toc = X_toc.apply(lambda x: ' '.join(x)).tolist()
share = int(len(X_toc)*.6)
X_train, Y_train = X_toc[:share], criterias_group[:share]
X_test, Y_test = X_toc[share:], criterias_group[share:]



x_train_shuf, x_test_shuf, y_train_shuf, y_test_shuf = train_test_split(X_train, Y_train.values, 
                                                                        test_size = 0.3, random_state = 0)

In [20]:
x_train_shuf = np.array(x_train_shuf, dtype = str)
x_test_shuf = np.array(x_test_shuf, dtype = str)

**ELMO deeppavlov**

In [ ]:
elmo_model = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz", trainable=True)
sess = tf.Session()

K.set_session(sess)
# Initialize sessions
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [42]:
a.shape

(2, 3, 1024)

In [22]:
class KerasLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(KerasLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.

        # These are the 3 trainable weights for word_embedding, lstm_output1 and lstm_output2
        self.kernel1 = self.add_weight(name='kernel1',
                                       shape=(3,),
                                      initializer='uniform',
                                      trainable=True)
        # This is the bias weight
        self.kernel2 = self.add_weight(name='kernel2',
                                       shape=(),
                                      initializer='uniform',
                                      trainable=True)
        super(KerasLayer, self).build(input_shape)

    def call(self, x):
        # Get all the outputs of elmo_model
        model =  elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)
        
        # Embedding activation output
        activation1 = model["word_emb"]
        
        # First LSTM layer output
        activation2 = model["lstm_outputs1"]
        
        # Second LSTM layer output
        activation3 = model["lstm_outputs2"]

        activation2 = tf.reduce_mean(activation2, axis=1)
        activation3 = tf.reduce_mean(activation3, axis=1)
        
        mul1 = tf.scalar_mul(self.kernel1[0], activation1)
        mul2 = tf.scalar_mul(self.kernel1[1], activation2)
        mul3 = tf.scalar_mul(self.kernel1[2], activation3)
        
        sum_vector = tf.add(mul2, mul3)
        
        return tf.scalar_mul(self.kernel2, sum_vector)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [23]:
earlystop = EarlyStopping(monitor='value_loss')
callbacks_list = [earlystop]


input_text = Input(shape=(1,), dtype=tf.string)
custom_layer = KerasLayer(output_dim=1024, trainable=True)(input_text)
main_output = Dense(46, activation='softmax', name='main_output')(custom_layer)

model = Model(inputs = input_text,  outputs=main_output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

history = model.fit(x_train_shuf, y_train_shuf, validation_data = (x_test_shuf, y_test_shuf), \
                    batch_size=64, epochs=15, shuffle = True, callbacks=callbacks_list, verbose=1)

2019-01-18 16:10:18.219 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
keras_layer_1 (KerasLayer)   (None, 1024)              4         
_________________________________________________________________
main_output (Dense)          (None, 46)                47150     
Total params: 47,154
Trainable params: 47,154
Non-trainable params: 0
_________________________________________________________________
Train on 2583 samples, validate on 1108 samples
Epoch 1/15
2583/2583 [==============================] - ETA: 6:14 - loss: 3.8287 - acc: 0.015 - ETA: 3:10 - loss: 3.8282 - acc: 0.015 - ETA: 2:08 - loss: 3.8276 - acc: 0.041 - ETA: 1:36 - loss: 3.8269 - acc: 0.058 - ETA: 1:18 - loss: 3.8262 - acc: 0.087 - ETA: 1:05 - loss: 3.8256 - acc: 0.106 - ETA: 56s - loss: 3.8247 - acc: 0.129 - ETA: 49s - loss: 3.

Epoch 10/15
2583/2583 [==============================] - ETA: 14s - loss: 1.2514 - acc: 0.73 - ETA: 14s - loss: 1.2947 - acc: 0.71 - ETA: 13s - loss: 1.1868 - acc: 0.72 - ETA: 13s - loss: 1.2524 - acc: 0.71 - ETA: 13s - loss: 1.2856 - acc: 0.70 - ETA: 12s - loss: 1.2852 - acc: 0.70 - ETA: 12s - loss: 1.2723 - acc: 0.70 - ETA: 12s - loss: 1.2910 - acc: 0.70 - ETA: 11s - loss: 1.2950 - acc: 0.70 - ETA: 11s - loss: 1.3057 - acc: 0.70 - ETA: 10s - loss: 1.3064 - acc: 0.70 - ETA: 10s - loss: 1.2749 - acc: 0.70 - ETA: 10s - loss: 1.2592 - acc: 0.71 - ETA: 9s - loss: 1.2854 - acc: 0.7076 - ETA: 9s - loss: 1.2959 - acc: 0.705 - ETA: 9s - loss: 1.2905 - acc: 0.706 - ETA: 8s - loss: 1.2912 - acc: 0.704 - ETA: 8s - loss: 1.2872 - acc: 0.704 - ETA: 7s - loss: 1.2908 - acc: 0.705 - ETA: 7s - loss: 1.2917 - acc: 0.706 - ETA: 7s - loss: 1.2938 - acc: 0.701 - ETA: 6s - loss: 1.2930 - acc: 0.701 - ETA: 6s - loss: 1.3079 - acc: 0.702 - ETA: 6s - loss: 1.3101 - acc: 0.703 - ETA: 5s - loss: 1.3059 - acc: 

In [26]:
X_test = np.array(X_test, dtype = str)
pred_values = model.predict(X_test)

In [28]:
predict_test_classes = pred_values.argmax(axis=-1)

In [31]:
accuracy_score(Y_test.values.argmax(-1), predict_test_classes)

0.6347013409183259

In [13]:
def create_elmo_embeddings(elmo, documents, max_sentences = 1000):
    num_sentences = min(max_sentences, len(documents)) if max_sentences > 0 else len(documents)
    print("\n\n:: Lookup of "+str(num_sentences)+" ELMo representations. This takes a while ::")
    embeddings = []
    labels = []
    tokens = [document['tokens'] for document in documents]
    
    documentIdx = 0
    for elmo_embedding in elmo.embed_sentences(tokens):  
        document = documents[documentIdx]
        # Average the 3 layers returned from ELMo
        avg_elmo_embedding = np.average(elmo_embedding, axis=0)
             
        embeddings.append(avg_elmo_embedding)        
        labels.append(document['label'])
            
        # Some progress info
        documentIdx += 1
        percent = 100.0 * documentIdx / num_sentences
        line = '[{0}{1}]'.format('=' * int(percent / 2), ' ' * (50 - int(percent / 2)))
        status = '\r{0:3.0f}%{1} {2:3d}/{3:3d} sentences'
        sys.stdout.write(status.format(percent, line, documentIdx, num_sentences))
        
        if max_sentences > 0 and documentIdx >= max_sentences:
input_text = Input(shape=(1,), dtype=tf.string)
custom_layer = KerasLayer(output_dim=1024, trainable=True)(input_text)
pred = Dense(1, activation='sigmoid', trainable=True)(custom_layer)

model = Model(inputs=input_text, outputs=pred)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(train_text, y_train, epochs=15, batch_size=32)            break
            
    return embeddings, labels

'/device:GPU:0'

In [40]:
tokens_input = [["мама", "мыла", "раму"], ["рама", "", ""]]
tokens_length = [3, 1]
embeddings = elmo_model(inputs={
                        "tokens": tokens_input,
                        "sequence_len": tokens_length
                        },
                signature="tokens",
                as_dict=True)["elmo"]
a = sess.run(embeddings)

2019-01-18 16:40:00.82 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


In [ ]:
X_toc = data_events['CLEAN_TEXT'].apply(lambda x: x[:45])

share = int(len(X_toc)*.6)
X_train, Y_train = X_toc[:share], criterias_group[:share]
X_test, Y_test = X_toc[share:], criterias_group[share:]



x_train_shuf, x_test_shuf, y_train_shuf, y_test_shuf = train_test_split(X_train, Y_train.values, 
                                                                        test_size = 0.3, random_state = 0)